<a href="https://colab.research.google.com/github/chaeyeon573/Sales-Analysis/blob/main/Project_Sales_Analysis_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요 라이브러리 호출하기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('default')

In [ ]:
# 데이터 호출하기
entry_data = pd.read_csv('/aiffel/data/entry_data.csv')
sales_data = pd.read_csv('/aiffel/data/sales_data.csv')
item_data = pd.read_csv('/aiffel/data/item_data.csv')
item_entry_data = pd.read_csv('/aiffel/data/item_entry_data.csv')

In [ ]:
# entry_data 데이터 정보 확인하기
entry_data.head()

In [ ]:
item_data.head()

In [ ]:
# 문제 1: 월별 매출액 추세
# 활용할 데이터(sales_data)를 temp_var로 정의하기
# [[YOUR CODE]]

temp_var = sales_data

# temp_var의 date열을 만들고, sales_date의 결제일시 중 앞에서 7번째 값까지 할당
# [[YOUR CODE]]

temp_var["date"] = temp_var['결제일시'].str[:7]

temp_var

In [ ]:
# sales_data의 결제금액에 포함된 쉼표(,) 지우기
# [[YOUR CODE]]
sales_data['결제금액'] = sales_data['결제금액'].replace(",","", regex = True)

# sales_data의 결제금액을 숫자형태로 바꾸기
# [[YOUR CODE]]

sales_data['결제금액'] = sales_data['결제금액'].astype(int)
sales_data

# temp_var의 payment열을 date 기준으로 집계하기
# [[YOUR CODE]]

payment_data = temp_var['결제금액'].groupby(temp_var["date"]).sum()
payment_data.reset_index(drop = False)

In [ ]:
# 집계 결과를 시각화하기
# [[YOUR CODE]]

# payment_data.plot()
payment_data.rename(columns = {"결제금액":'payment'}, inplace = True)
payment_data
plt.figure(figsize = (15,5))
ax = sns.barplot(x = payment_data["date"], y = payment_data['payment'])



container = ax.containers[0]

ax.bar_label( container, labels = [f'{x:,.0f}' for x in container.datavalues],label_type = "center")


plt.title("payment by month")
plt.xlabel("date")
plt.ylabel("won")

In [ ]:
# 문제 2: 가장 인기 있는 음식/음료
# temp_var의 상품명을 value_counts를 이용해 집계하기
# 주의! value_counts로 바로 집계하면 판매취소와 결제취소가 함께 집계됩니다.
# 따라서 '판매수량' 칼럼 값이 '-1'인 데이터를 제외하고 집계해야 합니다.
# 힌트 코드 : temp_var[temp_var['판매수량']==-1]
# [[YOUR CODE]]

# 집계 결과를 데이터프레임으로 변환하고, 상위 10개 선정하기
# [[YOUR CODE]]


temp_var = sales_data
temp_var = temp_var[temp_var['판매수량']!=-1].copy()
temp_var

temp_var.loc[:,'실결제금액'] = temp_var['판매수량']*temp_var['결제금액']
temp_var[temp_var['결제금액']!=temp_var['실결제금액']]
temp_var


In [ ]:
temp_var_sales = temp_var.groupby('상품명')['결제금액'].sum()
temp_var_sales = temp_var_sales.sort_values( ascending = False)
temp_var_sales = temp_var_sales.reset_index(drop = False)
temp_var_sales

product_price = temp_var.groupby('결제금액')['상품명'].sum()
product_price = product_price.sort_values(ascending = False)
product_price =  product_price.reset_index(drop = False)
temp_var.drop_duplicates(subset = '상품명',inplace = True, keep = 'first' )
print(temp_var)
temp_var.info()

# # print(type(temp_var_sales))
unique_values = temp_var_sales[['상품명','결제금액']].unique()
# print(len(unique_values))

In [ ]:
# 한글이 포함된 시각화를 위해 Matplotlib 설정
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic')
mpl.font_manager.findfont(font)
print("완료!")

In [ ]:
# 집계 결과를 시각화하기

import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 준비
best_10 = temp_var_sales[:10]  # 최다 매출 상품 10개
plt.figure(figsize=(15, 6))

# 막대그래프 생성
ax = sns.barplot(data=best_10, x="상품명", y="결제금액")
container = ax.containers[0]

# 상품명과 금액을 함께 표시
ax.bar_label(
    container,
    labels=[
        f'{product}\n매출: {value:,.0f}원\n상품가: {price:,.0f}'
        for product, value, price in zip(best_10['상품명'], container.datavalues,temp_var['결제금액'].to_list())
    ],
    label_type="center"
)

# 그래프 꾸미기
plt.xlabel("상품")
plt.ylabel("결제금액")
plt.title("최다 매출 상품 10개")
plt.xticks(rotation=45)  # 상품명이 겹치지 않게 회전
plt.tight_layout()
plt.show()


In [ ]:
# 문제 3: 가장 많이 팔린 입장권 종류
# 활용할 데이터(entry_data)를 temp_var로 정의하기
# [[YOUR CODE]]
temp_var = entry_data
print(temp_var)

# temp_var의 금액을 숫자형태로 변환하기
# [[YOUR CODE]]
pd.to_numeric(temp_var['금액'])
temp_var.drop_duplicates(subset = None, keep = "first", inplace = True )
print(temp_var)
sales_by_product = temp_var.groupby("요금제명").agg({"금액":["sum","mean"]}).reset_index()

sales_by_product.columns = ['요금제명',"매출","단가"]
print(sales_by_product)

sales_by_product = sales_by_product.reset_index(drop=False)
sales_by_product.sort_values(by = "매출", ascending= False, inplace = True)
top_10 = sales_by_product[:10]
# temp_var의 금액을 요금제명을 기준으로 집계하기
# [[YOUR CODE]]

# top_10 = top_10.reset_index(drop = False)
# # top_10['상품가격'] = sales

plt.figure(figsize = (15,6))
ax  =  sns.barplot( data = top_10, x = "요금제명", y = "매출")
container = ax.containers[0]
ax.bar_label(container, labels = [f'\n{sales:,.0f}원\n{price:,.0f}원' for  sales, price in zip(container.datavalues, top_10["단가"])])
print(top_10.columns)
plt.xlabel("요금제명")
plt.ylabel("금액")
plt.show()


월별 매출액 추세: 2021년 6월이 가장 높은 매출액을, 2021년 9월이 가장 낮은 매출액을 보인다.
가장 인기있는 음식/음료: 김치볶음밥, 그리고 라면이 가장 많이 팔렸다.
가장 많이 팔린 입장권 종류: 한국대생(종일권)이 가장 많이 팔렸다. 두번째는 일반 입장권(종일권)이다

식품 가격의 변동이 필요하다. (예, 불닭볶음면의 가격 인상 ) 여름에 손님이 적으니 여름철 독서실의 실내환경을 점검해보는 것이 좋겠다. (에어컨을 더 많이 튼다던지 등)